In [0]:
sc


SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("PysparkProjectAnalysis").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

sqlcontext = SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
logs_DF = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv", header=True, inferSchema=True)

In [0]:
logs_DF.show(10)

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|
|7792c28f-2f9b-48b...|Adult Womens Sexy...|       adult|   shoes|  a

In [0]:
from pyspark.sql.functions import *

In [0]:
logs_DF.createTempView("AmazonTable")

In [0]:
spark.sql("select * from AmazonTable where SubCategory = 'mobile'").show(50)

+--------------------+--------------------+----------+-----------+-----------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|         ProductCode|         Description|     Brand|   Category|SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|
+--------------------+--------------------+----------+-----------+-----------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|75c1100b-92f2-416...|Apple iPhone 6S 1...|     apple|electronics|     mobile|    38430.0|              10|                10|       3|         103761.0|2015-01-04 18:30:00| dinner's club|   1045680714|0JxPng6IJEXW|    Japan|
|f2fa233f-78ea-4c4...|BLU Advance 4.0L ...|       blu|electronics|     mobile|    12600.0|      

##### 1 a)Find the top five performing mobile brands for the months of November, December and January r each year.

In [0]:
spark.sql("create table PhoneBrandtable1 as select InvoiceDate, Brand, Category, SubCategory, Quantity from AmazonTable where SubCategory='mobile' and MONTH(DATE(InvoiceDate)) in (01,11,12)")

Out[30]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("create table brandTable1 select YEAR(DATE(InvoiceDate)) as yearOfSale, MONTH(DATE(InvoiceDate)) as monthOfSale, Brand, sum(Quantity) as quantity from PhoneBrandTable1 group by Brand,InvoiceDate order by Brand").show(100)

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
+-----------------+-----------------+



In [0]:
spark.sql("select * from brandTable1 order by yearOfSale, monthOfSale").show()

+----------+-----------+-------+--------+
|yearOfSale|monthOfSale|  Brand|quantity|
+----------+-----------+-------+--------+
|      2015|          1|  apple|      13|
|      2015|          1|  black|       3|
|      2015|          1|  apple|      39|
|      2015|          1|   32gb|       2|
|      2015|          1|  apple|      35|
|      2015|          1|   acer|       2|
|      2015|          1|  apple|       6|
|      2015|          1|   aeku|       1|
|      2015|          1|  apple|       6|
|      2015|          1|alcatel|       3|
|      2015|          1|  apple|       5|
|      2015|          1|   andy|       2|
|      2015|          1|  apple|       7|
|      2015|          1|  apple|       7|
|      2015|          1|  apple|      31|
|      2015|          1|  apple|      29|
|      2015|          1|   at&t|       4|
|      2015|          1|  argom|       2|
|      2015|          1|   acer|       4|
|      2015|          1|  argom|       2|
+----------+-----------+-------+--

In [0]:
spark.sql("create table ymSaleTable as select yearOfSale, monthOfSale, Brand, sum(quantity) as quantity from brandTable where yearOfSale=2015 and monthOfsale = 1 group by monthOfSale, yearOfSale, Brand order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
+-----------------+-----------------+



In [0]:
spark.sql("select * from ymSaleTable").show()

+----------+-----------+--------+--------+
|yearOfSale|monthOfSale|   Brand|quantity|
+----------+-----------+--------+--------+
|      2015|          1| samsung|     377|
|      2015|          1|   apple|     223|
|      2015|          1|   nokia|     168|
|      2015|          1|     blu|     148|
|      2015|          1|motorola|     127|
+----------+-----------+--------+--------+



In [0]:
spark.sql("Insert into ymSaleTable(yearOfSale, monthOfSale, Brand, quantity) select yearOfSale, monthOfSale, Brand, sum(quantity) as quantity from brandTable where yearOfSale=2015 and monthOfsale = 11 group by monthOfSale, yearOfSale, Brand order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+



In [0]:
spark.sql("select * from ymSaleTable").show()

+----------+-----------+--------+--------+
|yearOfSale|monthOfSale|   Brand|quantity|
+----------+-----------+--------+--------+
|      2015|          1| samsung|     377|
|      2015|          1|   apple|     223|
|      2015|          1|   nokia|     168|
|      2015|          1|     blu|     148|
|      2015|          1|motorola|     127|
|      2015|         11| samsung|     448|
|      2015|         11|     blu|     234|
|      2015|         11|   apple|     200|
|      2015|         11|   nokia|     185|
|      2015|         11|      lg|     165|
+----------+-----------+--------+--------+



In [0]:
spark.sql("Insert into ymSaleTable(yearOfSale, monthOfSale, Brand, quantity) select yearOfSale, monthOfSale, Brand, sum(quantity) as quantity from brandTable where yearOfSale=2015 and monthOfsale = 12 group by monthOfSale, yearOfSale, Brand order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+



In [0]:
spark.sql("Insert into ymSaleTable(yearOfSale, monthOfSale, Brand, quantity) select yearOfSale, monthOfSale, Brand, sum(quantity) as quantity from brandTable where yearOfSale=2018 and monthOfsale = 12 group by monthOfSale, yearOfSale, Brand order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                0|                0|
+-----------------+-----------------+



##### 1 a)Find the top five performing mobile brands for the months of November, December and January r each year.

In [0]:
spark.sql("select * from ymSaleTable order by yearOfSale, monthOfSale, Quantity desc").show(60)

+----------+-----------+--------+--------+
|yearOfSale|monthOfSale|   Brand|quantity|
+----------+-----------+--------+--------+
|      2015|          1| samsung|     377|
|      2015|          1|   apple|     223|
|      2015|          1|   nokia|     168|
|      2015|          1|     blu|     148|
|      2015|          1|motorola|     127|
|      2015|         11| samsung|     448|
|      2015|         11|     blu|     234|
|      2015|         11|   apple|     200|
|      2015|         11|   nokia|     185|
|      2015|         11|      lg|     165|
|      2015|         12| samsung|     517|
|      2015|         12|     blu|     249|
|      2015|         12|   nokia|     212|
|      2015|         12|   apple|     184|
|      2015|         12|     htc|     158|
|      2016|          1| samsung|     361|
|      2016|          1|   apple|     216|
|      2016|          1|     blu|     171|
|      2016|          1|   nokia|     159|
|      2016|          1|      lg|     145|
|      2016

1b)Find the top five selling products for the months of November, December and January for each year.

In [0]:
spark.sql("create table Producttable as select InvoiceDate, ProductCode, Description, Category, Quantity from AmazonTable where MONTH(DATE(InvoiceDate)) in (01,11,12)")

Out[90]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select * from Producttable").show()

+-------------------+--------------------+--------------------+--------+--------+
|        InvoiceDate|         ProductCode|         Description|Category|Quantity|
+-------------------+--------------------+--------------------+--------+--------+
|2015-01-04 18:30:00|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|   shoes|       3|
|2015-01-04 18:30:00|7792c28f-2f9b-48b...|Adult Womens Sexy...|   shoes|       2|
|2015-01-04 18:30:00|7c478913-348f-4f3...|Woolrich Miss Luc...|clothing|       3|
|2015-01-04 18:30:00|cf9b6432-8b56-42c...|Style & Co Vern  ...|clothing|       2|
|2015-01-04 18:30:00|05eae2f8-0c6a-47c...|Two Lips Swizzle ...|clothing|       3|
|2015-01-04 18:30:00|e8ffc821-8e90-49d...|Michael Kors Clar...|clothing|       2|
|2015-01-04 18:30:00|28a330c4-23c5-4bb...|Bali 3547 Womens ...|clothing|       2|
|2015-01-04 18:30:00|e0cbbd76-d953-479...| Women's Dr Scholl's|clothing|       3|
|2015-01-04 18:30:00|6ffddd0f-457e-450...|Coloriffics Danic...|clothing|       4|
|2015-01-04 18:3

In [0]:
spark.sql("create table pTable select YEAR(DATE(InvoiceDate)) as yearOfSale, MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as quantity from Producttable group by Description,InvoiceDate order by Description").show(100)

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
+-----------------+-----------------+



In [0]:
spark.sql("select * from pTable").show()

+----------+-----------+--------------------+--------+
|yearOfSale|monthOfSale|         Description|quantity|
+----------+-----------+--------------------+--------+
|      2017|         12|#brand New# Calla...|       4|
|      2015|         11|$125 Brand New - ...|       4|
|      2015|         11|$125 Brand New - ...|       4|
|      2017|          1|$125 Brand New - ...|       4|
|      2016|         11|$125 Brand New - ...|       4|
|      2018|          1|$125 Lauren Ralph...|       1|
|      2017|         12|$125 Lauren Ralph...|       1|
|      2017|          1|$125 Lauren Ralph...|       1|
|      2016|          1|$125 Lauren Ralph...|       1|
|      2016|          1|$125 Lauren Ralph...|       1|
|      2015|          1|$125 Lauren Ralph...|       1|
|      2015|         11|$125 Nwt Nautica ...|       3|
|      2017|         12|$125 Nwt Nautica ...|       3|
|      2017|          1|$125 Nwt Nautica ...|       3|
|      2015|         12|$125 Nwt Nautica ...|       6|
|      201

In [0]:
spark.sql("drop table ProductSaleTable")

Out[102]: DataFrame[]

In [0]:
spark.sql("create table ProductSaleTable as select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where yearOfSale=2015 and monthOfsale = 1 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
+-----------------+-----------------+



In [0]:
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where yearOfSale=2015 and monthOfsale = 11 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5")
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where yearOfSale=2015 and monthOfsale = 12 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5")
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2016 and monthOfsale = 1 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5")
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2016 and monthOfsale = 11 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5")
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2016 and monthOfsale = 12 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5")
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where yearOfSale=2017 and monthOfsale = 1 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2017 and monthOfsale = 11 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2017 and monthOfsale = 12 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2018 and monthOfsale = 1 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale,quantity desc limit 5").show()
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where  yearOfSale=2018 and monthOfsale = 11 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()
spark.sql("Insert into ProductSaleTable(yearOfSale, monthOfSale, Description, quantity) select yearOfSale, monthOfSale, Description, sum(quantity) as quantity from pTable where yearOfSale=2018 and monthOfsale = 12 group by monthOfSale, yearOfSale, Description order by yearOfSale, monthOfSale, quantity desc limit 5").show()

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                5|                5|
+-----------------+-----------------+

+-----------------+-----------------+
|num_affected_rows|num_inserted_rows|
+-----------------+-----------------+
|                0|                0|
+-----------------+-----------------+

+-----------------+-----------------+
|num_af

##### 1b)Find the top five selling products for the months of November, December and January for each year.

In [0]:
spark.sql("select * from ProductSaleTable order by yearOfSale, monthOfSale, Quantity desc").show(60)

+----------+-----------+--------------------+--------+
|yearOfSale|monthOfSale|         Description|quantity|
+----------+-----------+--------------------+--------+
|      2015|          1|Nina Women's Mari...|      12|
|      2015|          1|OnePlus 2 Unlocke...|      12|
|      2015|          1|Reebok Pro Evalua...|      12|
|      2015|          1|Topbest Battery C...|      12|
|      2015|          1|Shoes of Soul Wom...|      12|
|      2015|         11|Vans Unisex Sk8-h...|      12|
|      2015|         11|Nokia 2700 Classi...|      12|
|      2015|         11|Nike Mens Air Foa...|      12|
|      2015|         11|verykool Cyprus L...|      12|
|      2015|         11|Propper Mens Coyo...|      12|
|      2015|         12|Nike Women's Flex...|      16|
|      2015|         12|Yu&yu Women's Sho...|      16|
|      2015|         12|Jessica Simpson B...|      16|
|      2015|         12|HTC One E9 Plus d...|      16|
|      2015|         12|Style & Co Tiffan...|      12|
|      201

#####2. Give valuable insights to the investors

#####a. Number of products sold along with revenue aggregated quarterly

In [0]:
spark.sql("select Quarter(InvoiceDate) as quarterer, YEAR(DATE(InvoiceDate)) as yearOfSale, Description, count(Quantity) as productSold, sum(TransactionAmount) as Revenue from AmazonTable group by quarterer,Description, yearOfSale order by Revenue desc").show(60)

+---------+----------+--------------------+-----------+-------------+
|quarterer|yearOfSale|         Description|productSold|      Revenue|
+---------+----------+--------------------+-----------+-------------+
|        3|      2016|Pioneer Avh-x3800...|          3|4.83210006E14|
|        1|      2018|Pioneer Avh-x3800...|          3|4.83210006E14|
|        4|      2016|Pioneer Avh-x3800...|          2|3.22140004E14|
|        4|      2017|Pioneer Avh-x3800...|          2|3.22140004E14|
|        2|      2016|Pioneer Avh-x3800...|          2|3.22140004E14|
|        2|      2015|Pioneer Avh-x3800...|          1|1.61070002E14|
|        1|      2016|Pioneer Avh-x3800...|          1|1.61070002E14|
|        4|      2017|Annie Shoes 6200 ...|          2|    1.16872E7|
|        1|      2016|Aquaracer Mens Watch|          3|    7206738.0|
|        2|      2017|Nike Womens Flykn...|          2|    6543852.0|
|        3|      2015|Nike Womens Flykn...|          2|    6543852.0|
|        2|      201

##### 2.b) No of product sold along with category aggregated quaterly

In [0]:
spark.sql("select Quarter(InvoiceDate) as quarterer, YEAR(DATE(InvoiceDate)) as yearOfSale, Category, count(Quantity) as productSold, sum(TransactionAmount-ActualPrice) as Revenue from AmazonTable group by quarterer,Category, yearOfSale order by productSold desc").show(60)

+---------+----------+-------------+-----------+-------------------+
|quarterer|yearOfSale|     Category|productSold|            Revenue|
+---------+----------+-------------+-----------+-------------------+
|        1|      2016|     clothing|      15340|       1.15214575E8|
|        4|      2017|     clothing|      13890|       1.12093765E8|
|        1|      2017|     clothing|      13621|       1.00434327E8|
|        3|      2016|     clothing|      13358|        9.7926038E7|
|        4|      2015|     clothing|      13353|        9.0226346E7|
|        2|      2017|     clothing|      13244|       1.03454897E8|
|        3|      2015|     clothing|      13201|       1.03834724E8|
|        1|      2018|     clothing|      13050|        9.5143986E7|
|        2|      2015|     clothing|      12993|        9.2899422E7|
|        1|      2015|     clothing|      12961|        9.3573375E7|
|        4|      2016|     clothing|      12753|        9.3606247E7|
|        2|      2016|     clothin

##### 2c) Number of Product sold for each brand aggregated quaterly

In [0]:
spark.sql("select Quarter(InvoiceDate) as quarterer,YEAR(DATE(InvoiceDate)) as yearOfSale, Brand, sum(Quantity) as productSold, sum(TransactionAmount-ActualPrice) as Revenue from AmazonTable group by Brand,quarterer,yearOfSale order by productSold desc").show(60)

+---------+----------+-------+-----------+-----------+
|quarterer|yearOfSale|  Brand|productSold|    Revenue|
+---------+----------+-------+-----------+-----------+
|        1|      2016|   nike|       2902| 1.221997E7|
|        1|      2017|   nike|       2743|1.0546984E7|
|        2|      2017|   nike|       2731|1.4116333E7|
|        4|      2016|   nike|       2729| 1.150198E7|
|        4|      2017|   nike|       2645|1.0897222E7|
|        1|      2015|   nike|       2642|  7757197.0|
|        1|      2018|   nike|       2623|  8330112.0|
|        3|      2016|   nike|       2544|  7736470.0|
|        4|      2015|   nike|       2520|  7608783.0|
|        3|      2015|   nike|       2520|1.4418579E7|
|        2|      2016|   nike|       2399|1.0279059E7|
|        2|      2015|   nike|       2336|  7377734.0|
|        3|      2017|   nike|       2330|1.0392053E7|
|        4|      2015|samsung|       1536|1.3020315E7|
|        2|      2017|samsung|       1463|1.4624274E7|
|        3

##### 3.a) Top 5 profit margin generating percentage

In [0]:
spark.sql("select Description, sum(Marginpercentage) as Profit, YEAR(DATE(InvoiceDate)) as yearOfSale, MONTH(DATE(InvoiceDate)) as monthOfSale from AmazonTable group by Description, InvoiceDate, monthOfSale order by Profit desc").show()

+--------------------+------+----------+-----------+
|         Description|Profit|yearOfSale|monthOfSale|
+--------------------+------+----------+-----------+
|Us Men's Vintage ...|    60|      2015|          2|
|Leica DG Vario-El...|    60|      2016|          2|
|Marc Fisher Hien ...|    40|      2017|          7|
|Samsung Galaxy J2...|    40|      2015|          3|
|A2 By Aerosoles B...|    40|      2015|          4|
|4G-Unlocked Huawe...|    40|      2015|         10|
|Allrounder By Mep...|    40|      2015|          4|
|David Donahue Men...|    40|      2015|          1|
|Size 8-1/2 Work B...|    40|      2015|          8|
|Seek Optics Seek ...|    40|      2016|         11|
|Women's Ara Rosal...|    40|      2015|         12|
|'betty' Slipper (...|    40|      2016|          2|
|Bcbgeneration Kae...|    40|      2016|          8|
|Norcross Safety P...|    40|      2016|          5|
|Yu&yu Women's Sho...|    40|      2016|          8|
|New Oakley Sungla...|    40|      2016|      

##### 4. a)Brand wise seggregation of best performing categories profit wise

In [0]:
spark.sql("select Brand, sum(TransactionAmount) as profits from AmazonTable group by Brand order by profits desc").show(60)

+----------+--------------------+
|     Brand|             profits|
+----------+--------------------+
|   pioneer|2.254980029415246E15|
|   samsung|        1.56318183E8|
|      nike|        1.33997472E8|
|     apple|         8.3756036E7|
|      sony|         3.9833647E7|
|       blu|         3.6263759E7|
|     annie|         3.5354872E7|
|     nokia|         2.8565852E7|
|        lg|         2.6923484E7|
| aquaracer|         2.6424706E7|
|       htc|         2.4492237E7|
|      sexy|         2.4031903E7|
|     yu&yu|         2.3679124E7|
|    reebok|           2.36621E7|
|      puma|         2.1785295E7|
|       new|         1.9895974E7|
|      vans|         1.7877734E7|
|    yamaha|         1.6978136E7|
|    adidas|         1.6823695E7|
|  motorola|         1.5252657E7|
|pleaserusa|         1.4451073E7|
|  twobirch|         1.4223076E7|
|    huawei|         1.4016541E7|
|blackberry|         1.3363959E7|
|      nine|         1.2830601E7|
|  skechers|         1.1358102E7|
|      toms|  

##### 4. b)Brand wise seggregation of least performing categories order wise

In [0]:
spark.sql("select Brand, sum(Quantity) as orders from AmazonTable group by Brand order by orders").show(60)

+--------------------+------+
|               Brand|orders|
+--------------------+------+
|              cooler|     1|
|              oregon|     1|
|              hailey|     1|
|    jeffrey campbell|     1|
|                idea|     1|
|           muckboots|     1|
|             elliott|     1|
|           calibrate|     1|
|              castle|     1|
|             glister|     1|
|              cliffs|     1|
|               irish|     1|
|                 riv|     1|
|               knife|     1|
|                loro|     1|
|           superfeet|     1|
|             luckers|     1|
|              dabuyu|     1|
|          highstreet|     1|
|                doro|     1|
|           stainless|     1|
|              kendal|     1|
|                male|     1|
|            colorado|     1|
|                  fq|     1|
|                 his|     1|
|           bonnnibel|     1|
|               coral|     1|
|              parisa|     1|
|               kathy|     2|
|         

##### 5. a)find the top 5 selling products in the given month

In [0]:
spark.sql("create table monthProductSelling as select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 1 group by monthOfSale, Description order by QuantitySold desc limit 5")

Out[8]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 2 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 3 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 4 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 5 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 6 group by monthOfSale, Description order by QuantitySold desc limit 5")

Out[10]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 7 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 8 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 9 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 10 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 11 group by monthOfSale, Description order by QuantitySold desc limit 5")
spark.sql("insert into monthProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 12 group by monthOfSale, Description order by QuantitySold desc limit 5")

Out[11]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from monthProductSelling order by monthOfSale,QuantitySold;

monthOfSale,Description,QuantitySold
1,Asics Gel-nimbus 16 Women's Us Sz 7 Iridescent Pink Capri Blue T485q 0235,24
1,LG Xenon GR500 Unlocked Phone with QWERTY Keyboard,24
1,Intaglia Designs Women's Nashville Black Extended-calf Knee-high Fashion Riding Boot - Wide Width Available,24
1,Alpinestars Fraction Sandals Flip Flops Red/black,28
1,Sony Xperia X Compact - Unlocked Smartphone - 32GB - Black (US Warranty),28
2,Carrini Ca Collection Women's Fashion Knit Sweater Cuff and Foldover Lace-up Booties,24
2,Mountain Hardwear Men's Monkey Man Grid Ii Jacket,24
2,Sexy Satin Crotchless Lace Back Frilly Knickers 6-24 Inc Plus Sizes Black White,24
2,Dc Comics Batman Batmobile Slippers (medium 7/8),24
2,Lauren Ralph Lauren Women's Caliana-fl-csl Leather Porcini Ankle-high Leather Flat Shoe - 7.5m,24


##### 5. b)top 5 products of the having the highest margin in a given month

In [0]:
spark.sql("create table monthProductMargin as select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 1 group by monthOfSale, Description order by SaleMargin desc limit 5")

Out[12]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 2 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 3 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 4 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 5 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 6 group by monthOfSale, Description order by SaleMargin desc limit 5")

Out[16]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 7 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 8 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 9 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 10 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 11 group by monthOfSale, Description order by SaleMargin desc limit 5")
spark.sql("insert into monthProductMargin(monthOfSale, Description, SaleMargin) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Marginpercentage-Discountpercentage) as SaleMargin from AmazonTable where MONTH(DATE(InvoiceDate)) = 12 group by monthOfSale, Description order by SaleMargin desc limit 5")

Out[17]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select * from monthProductMargin order by monthOfSale, SaleMargin").show()

+-----------+--------------------+----------+
|monthOfSale|         Description|SaleMargin|
+-----------+--------------------+----------+
|          1|Emilio Pucci Ep11...|       120|
|          1|Inc Plus Matte Je...|       120|
|          1|Epic 7 Tablet - F...|       140|
|          1|BlackBerry Pearl ...|       140|
|          1|Fuse Lenses For R...|       140|
|          2|Puma Evopower 1.2...|       100|
|          2|Coleman Apparel F...|       100|
|          2|Coleman Apparel F...|       100|
|          2|Jewelry Adviser L...|       100|
|          2|Puma Evopower 1.2...|       100|
|          2|Samsung Note 5 N9...|       100|
|          2|Diesel Womens Wal...|       100|
|          2|Jewelry Adviser L...|       100|
|          2|Samsung Note 5 N9...|       100|
|          2|Diesel Womens Wal...|       100|
|          3|Roper Western Boo...|       120|
|          3|Berne Apparel Wjs...|       120|
|          3|Nike Men's Tiempo...|       140|
|          3|Huadoo V3 IP68 Ru...|

##### 5. c) Find the five lowest selling products in a given month.

In [0]:
spark.sql("create table lowestProductSelling as select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 1 group by monthOfSale, Description order by QuantitySold limit 5")

Out[20]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 2 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 3 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 4 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 5 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 6 group by monthOfSale, Description order by QuantitySold limit 5")

Out[22]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 7 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 8 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 9 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 10 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 11 group by monthOfSale, Description order by QuantitySold limit 5")
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 12 group by monthOfSale, Description order by QuantitySold limit 5")

Out[23]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("insert into lowestProductSelling(monthOfSale, Description, QuantitySold) select MONTH(DATE(InvoiceDate)) as monthOfSale, Description, sum(Quantity) as QuantitySold from AmazonTable where MONTH(DATE(InvoiceDate)) = 2 group by monthOfSale, Description order by QuantitySold limit 5")

Out[36]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select * from lowestProductSelling order by monthOfSale, QuantitySold").show(60)

+-----------+--------------------+------------+
|monthOfSale|         Description|QuantitySold|
+-----------+--------------------+------------+
|          1|Inc New Black Whi...|           1|
|          1|Ben Sherman Mens ...|           1|
|          1|Twenty Four Seven...|           1|
|          1|LG G4 Power Charg...|           1|
|          1|Samsung Galaxy J1...|           1|
|          2|BLU PURE XL Smart...|           1|
|          2|Dan Post Men's Dp...|           1|
|          2|Yu&yu Women's Sho...|           1|
|          2|verykool s3503 Un...|           1|
|          2|Mcr Safety P1d383...|           1|
|          3|BLU Dash X Plus -...|           1|
|          3|Motorola KRZR K1 ...|           1|
|          3|Unique Bargains 5...|           1|
|          3|'windsor' Slip-on...|           1|
|          3|Gold 5.0 Inch LCD...|           1|
|          4|New Balance Wx409...|           1|
|          4|MTX Audio - MTX 8...|           1|
|          4|Unique Bargains 3...|      